In [35]:
pip install tensorflow scikit-learn

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [36]:
from datasets import load_dataset

data = load_dataset("tasksource/crowdflower", "airline-sentiment")

In [37]:
data = data['train']

In [38]:
data = train_data.to_pandas()
print(data.head())

                                                text  label  idx
0                @VirginAmerica What @dhepburn said.      0    0
1  @VirginAmerica plus you've added commercials t...      1    1
2  @VirginAmerica I didn't today... Must mean I n...      0    2
3  "@VirginAmerica it's really aggressive to blas...      2    3
4  @VirginAmerica and it's a really big bad thing...      2    4


In [39]:
data.describe()

,label,idx
count,14640.000000,14640.000000
mean,1.415232,7406.997678
std,0.816223,4273.696204
min,0.000000,0.000000
25%,1.000000,3708.750000
50%,2.000000,7419.500000
75%,2.000000,11098.250000
max,2.000000,14809.000000


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14640 non-null  object
 1   label   14640 non-null  int64 
 2   idx     14640 non-null  int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 286.1+ KB


## Data Preprocessing

In [41]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Preprocess the text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = text.lower()
    return text

data['clean_text'] = data['text'].apply(clean_text)

# Split the data
X = data['clean_text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1
max_length = 100

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_length)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_length)


## Feature extraction using GloVe

In [42]:
# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glovetxt/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


## Building ABLSTM Model

In [43]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input, Dropout, Attention
from tensorflow.keras.models import Model

def create_ablstm_model(vocab_size, embedding_dim, max_length, embedding_matrix):
    input = Input(shape=(max_length,))
    x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)(input)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    attention = Attention()([x, x])
    x = tf.reduce_sum(attention, axis=1)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(3, activation='softmax')(x)

    model = Model(inputs=input, outputs=output)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_ablstm_model(vocab_size, embedding_dim, max_length, embedding_matrix)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 100, 100)             1354700   ['input_2[0][0]']             
                                                                                                  
 bidirectional_1 (Bidirecti  (None, 100, 128)             84480     ['embedding_1[0][0]']         
 onal)                                                                                            
                                                                                                  
 attention_1 (Attention)     (None, 100, 128)             0         ['bidirectional_1[0][0]'

## Optimization with ESGO Algorithm

In [49]:

def esgo_optimize(model, X_train, y_train, X_val, y_val, population_size=3, iterations=5):
    def initialize_population():
        population = []
        for _ in range(population_size):
            individual = {
                'learning_rate': np.random.uniform(0.0001, 0.01),
                'batch_size': np.random.randint(16, 128),
                'units': np.random.randint(32, 128)
            }
            population.append(individual)
        return population

    def fitness(individual):
         model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=individual['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

         history = model.fit(X_train, y_train,
                        epochs=3,  # Run for a few epochs to speed up iterations
                        batch_size=individual['batch_size'],
                        validation_data=(X_val, y_val),
                        verbose=0)

         val_accuracy = history.history['val_accuracy'][-1]
         return val_accuracy

    def update_position(individual, global_best, t, max_iterations):
        new_individual = {}
        for key in individual.keys():
            c1 = np.random.rand()
            c2 = np.random.rand()
            seagull_factor = np.random.rand()
            new_individual[key] = (individual[key] + c1 * (global_best[key] - individual[key]) +
                                   c2 * seagull_factor * (individual[key] - global_best[key]))
            # Ensure hyperparameters remain within valid ranges
            if key == 'learning_rate':
                new_individual[key] = np.clip(new_individual[key], 0.0001, 0.01)
            elif key == 'batch_size':
                new_individual[key] = np.clip(int(new_individual[key]), 16, 128)
            elif key == 'units':
                new_individual[key] = np.clip(int(new_individual[key]), 32, 128)
        return new_individual

    population = initialize_population()
    global_best = None
    global_best_fitness = -np.inf

    for t in range(iterations):
        for individual in population:
            current_fitness = fitness(individual)
            if current_fitness > global_best_fitness:
                global_best_fitness = current_fitness
                global_best = individual

        for i in range(population_size):
            population[i] = update_position(population[i], global_best, t, iterations)

        print(f'Iteration {t+1}/{iterations}, Best Accuracy: {global_best_fitness}')

    # Use the best hyperparameters found
    best_params = global_best
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model



In [45]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [46]:
X_train

array([[   0,    0,    0, ..., 1879,  564,  164],
       [   0,    0,    0, ...,  137,  650,  283],
       [   0,    0,    0, ..., 1173,    5,  238],
       ...,
       [   0,    0,    0, ...,   41,  190,   39],
       [   0,    0,    0, ...,   15,    2,  257],
       [   0,    0,    0, ...,    7,    4,  139]], dtype=int32)

In [47]:
y_train

6544     0
6707     2
3848     2
8299     0
8198     1
        ..
13503    0
10017    2
6691     2
12245    1
7361     0
Name: label, Length: 10540, dtype: int64

In [50]:
model = esgo_optimize(model, X_train, y_train, X_val, y_val)

W0000 00:00:1721038221.944087      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1721038226.986739      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key:

Iteration 1/5, Best Accuracy: 0.7977815866470337


W0000 00:00:1721038271.088809      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1721038275.555944      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key:

Iteration 2/5, Best Accuracy: 0.7977815866470337


W0000 00:00:1721038313.309749      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1721038317.486398      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key:

Iteration 3/5, Best Accuracy: 0.7977815866470337


W0000 00:00:1721038352.269207      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1721038356.445771      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key:

Iteration 4/5, Best Accuracy: 0.7977815866470337


W0000 00:00:1721038391.756987      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1721038395.595525      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key:

Iteration 5/5, Best Accuracy: 0.7977815866470337


## Model training and optimization

In [51]:
# Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')


Epoch 1/10


W0000 00:00:1721038449.065312      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


260/264 [============================>.] - ETA: 0s - loss: 0.1993 - accuracy: 0.9416

W0000 00:00:1721038453.463337      34 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla P100-PCIE-16GB" frequency: 1328 num_cores: 56 environment { key: "architecture" value: "6.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 4194304 shared_memory_size_per_multiprocessor: 65536 memory_size: 16266690560 bandwidth: 732160000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


264/264 [==============================] - 8s 15ms/step - loss: 0.1984 - accuracy: 0.9417 - val_loss: 0.1412 - val_accuracy: 0.9587
Epoch 2/10
264/264 [==============================] - 3s 12ms/step - loss: 0.1512 - accuracy: 0.9520 - val_loss: 0.1404 - val_accuracy: 0.9535
Epoch 3/10
264/264 [==============================] - 3s 12ms/step - loss: 0.1223 - accuracy: 0.9594 - val_loss: 0.1477 - val_accuracy: 0.9540
Epoch 4/10
264/264 [==============================] - 3s 12ms/step - loss: 0.1784 - accuracy: 0.9457 - val_loss: 0.1856 - val_accuracy: 0.9459
Epoch 5/10
264/264 [==============================] - 3s 12ms/step - loss: 0.1503 - accuracy: 0.9501 - val_loss: 0.2204 - val_accuracy: 0.9459
Epoch 6/10
264/264 [==============================] - 3s 12ms/step - loss: 0.1217 - accuracy: 0.9598 - val_loss: 0.1964 - val_accuracy: 0.9440
Epoch 7/10
264/264 [==============================] - 3s 12ms/step - loss: 0.1010 - accuracy: 0.9653 - val_loss: 0.2053 - val_accuracy: 0.9483
Epoch 8/10